# Time Difference of Arrival Based Target Pursuit: A Model Predictive Control Approach
**Presenter**: Anton Tolstonogov

**Division**: Dynamical Systems and Ocean Robotics Lab

**Course**: Dynamical System and Optimization

**Keywords**: MPC, Autonomous Surface Vehicle, TDoA localization, Target Pursuit

In [ ]:
# Jupyter auxiliary functions 2 (loading modules)
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

from pathlib import Path

# Special python library for displaying images
from IPython.display import Image

# Different modules for loading video depending on the environment
if not IN_COLAB:
    from IPython.display import Video
else:
    import moviepy.editor

def run_video(video_path:str):
    """
    Run embed video because the behavior in VSCode and Google colab is differ.

    Args:
    ----
        video_path (str): the path to the video.

    Returns:
    -------
        function: run video
    """
    if IN_COLAB:
        return moviepy.editor.ipython_display(str(video_path))
    else:
        return Video(url=video_path, height=700)

# Introduction
**Motivation**: tracking and monitoring of acoustically tagged marine animals (whales, dolphins, etc.) by groups of surface vehicles for ocean sustainability.

**Challange**: The only information we have is the time difference of arrival (TDoA) acoustic signal at the receivers installed on group of vehicles, emitted by a tag installed on the target.

**Long-term goal**: Find solution for the **SLAP** problem for group of autonomous surface vehicles (**trackers** later) to track underwater targets using TDoA measurements.

**SLAP**: **S**imultaneous **L**ocalization **A**nd **P**ursuit

**Aim of this work**: provide the first insight into the question of how to plan optimal motions for a set of two trackers to maximize the information available for the localization of a target based on TDoA-based measurements.

## Example of the ASVs Group

<img src="fig/asv.png" 
        alt="Picture" 
        width="800"
        style="display: block; margin: 0 auto" />

## Final Result

In [ ]:
file_path =Path(f'fig/cumulative trajectories case 1.mp4')
run_video(file_path)

## TDoA Preliminary Analysis

**TDOA** (Time Difference of Arrival) positioning is a passive technique to localize and track emitting objects by exploiting the difference of signal arrival times at multiple, spatially-separated receivers.
<img src="fig/hyberbolic_example_crop.png" 
        alt="Picture" 
        width="500" 
        height="500" 
        style="display: block; margin: 0 auto" />


## TDoA-based Measurement Model
The position of the target in the case of two receivers lies on the hyperbola given by:

\begin{equation*}
    \frac{(q_{x,k}^{\mathcal{H}})^2}{a^2} - \frac{(q_{y,k}^{\mathcal{H}})^2}{b^2} = 1,
\end{equation*}

where the semi-axes of the hyperbola $a$ and $b$ are given by

\begin{align*}
    a^2 &= (\frac{1}{2}DD_{1,2T})^2 \\
    b^2 &= (\frac{1}{2}||\mathbf{p}_k^{[1]} - \mathbf{p}_k^{[2]}||)^2 - a^2
\end{align*}

where:
* $\mathbf{q}_k^{\mathcal{H}} = [q_{x,k}^{\mathcal{H}}, q_{y,k}^{\mathcal{H}}]^\intercal$ is the possible position of the target at discrete time $k$ in the hyperbolic reference frame $\{\mathcal{H}\}$,
* $DD_k^{1,2T}$ is a time difference of signal arrival between two trackers at discrete time $k$,
* $||\mathbf{p}_k^{[1]} - \mathbf{p}_k^{[2]}||$ is the distance between two trackers.

In [ ]:
file_path=Path('fig/pattern 4.mp4')
run_video(file_path)

## MPC framework for Target Pursuit

<img src="fig/MPC decomposition scheme.png" 
        alt="Picture" 
        width="600"
        style="display: block; margin: 0 auto" />
        
* **Phase 1.** Optimal motion planning for the rotation center to follow the target based on target position and velocity information,
* **Phase 2.** Building desired circular trajectories for the trackers around the rotation center,
* **Phase 3.** optimal trackers' motion planning along the desired circular trajectories.

### Tracker Kinematic Model (Continuous)
The planar motion of tracker $i$ can be described by the simplified kinematic model of nonholonomic mobile robot.
It model well-suitable for standard SUV (Surface Unmanned vehicle) equipped only with one stern thruster.

In state-space form the model looks as follows
\begin{equation*}
    \dot{\mathbf{x}}^{[i]} =
    \begin{bmatrix}
        v^{[i]}\cos{\psi^{[i]}}\\
        v^{[i]}\sin{\psi^{[i]}}\\
        r^{[i]} \\
    \end{bmatrix}
    =
    \begin{bmatrix}
        \cos{\psi^{[i]}} & 0\\
        \sin{\psi^{[i]}} & 0\\
        0 & 1 \\
    \end{bmatrix}
    \begin{bmatrix}
        v^{[i]} \\
        r^{[i]}
    \end{bmatrix}
\end{equation*}

\begin{equation*}
    \mathbf{x}^{[i]} = 
    \begin{bmatrix}
        x^{[i]} \\ y^{[i]} \\ \psi^{[i]} \\
    \end{bmatrix},
    \mathbf{u}^{[i]} =
    \begin{bmatrix}
        v^{[i]} \\ r^{[i]} \\
    \end{bmatrix},
\end{equation*}
where
* $\mathbf{x}^{[i]}$ is the state vector and $\mathbf{u}^{[i]}$ is the input vector,
* $\mathbf{p}^{[i]}=[x^{[i]}, y^{[i]}]^\intercal$ is the position of tracker $i$,
* $\psi^{[i]}$ is its heading angle,
* $\mathbf{v}^{[i]} = [v^{[i]}, r^{[i]}]^\intercal \in \mathcal{V} ^{[i]}$ and is linear and angular speed, respectively.

### Tracker Kinematic Model (Discrete)

\begin{equation*}
    \mathbf{x}_{k+1}^{[i]}
    =
    \begin{bmatrix}
        x^{[i]}_{k+1} \\
        y^{[i]}_{k+1} \\
        \psi^{[i]}_{k+1} \\
    \end{bmatrix}
    =
    \begin{bmatrix}
        x^{[i]}_k \\
        y^{[i]}_k \\
        \psi^{[i]}_k \\
    \end{bmatrix}
    +
    \Delta T
    \begin{bmatrix}
        v^{[i]}_k\cos{\psi^{[i]}_k}\\
        v^{[i]}_k\sin{\psi^{[i]}_k}\\
        r^{[i]}_k \\
    \end{bmatrix}
\end{equation*}
where
* $k \in \mathbb{N}$ indexes discrete time instants 
* $\Delta T$ is the discretization period.

## **Phase 1.** Motion Planning for the Rotation Center
<img src="fig/MPC decomposition scheme. Phase 1.png" 
        alt="Picture" 
        width="600"
        style="display: block; margin: 0 auto" />

### Optimal Control Probem Formulation for Rotation Center, $\mathcal{OCP1}$

\begin{equation*}
    \min_{\bar{\mathbf{u}}^{[I]}, \bar{\mathbf{x}}^{[I]}} J_\text{track}^{[D1]} + J_\text{velocity}^{[D1]},
\end{equation*}

\begin{align*}
    J_\text{track}^{[D1]} &= \sum_{n=k}^{k+N-1} (\mathbf{x}^{[I]}_n - \bar{\mathbf{x}}^\text{tg}_n)Q(\mathbf{x}^{[I]}_n - \bar{\mathbf{x}}^\text{tg}_n)^\intercal, \\
    J_\text{velocity}^{[D1]} &= \sum_{n=k}^{k+N-1} (\mathbf{u}^{[I]}_n - \mathbf{u}^{[t]})R(\mathbf{u}^{[I]}_n - \mathbf{u}^{[t]})^\intercal,
\end{align*}

subject to
\begin{align*}
    \bar{\mathbf{x}}_{n+1}^{[I]} &= \mathbf{g_d} (\bar{\mathbf{x}}_{n}^{[I]}, \bar{\mathbf{u}}_{n}^{[I]}), \\ 
    \bar{\mathbf{x}}_{k}^{[I]} &= \mathbf{x}_k^{[I]}, \\
    \bar{\mathbf{u}}_{n}^{[I]} &\in \mathcal{V}^{[I]}
\end{align*}
for $n \in \{k,\ldots, k+N-1\}$, where
* $\bar{\mathbf{x}}^\text{tg}_n$ predicted position of the target under the linear assumption,
* $\mathbf{u}^{[t]}$ constant velocity of the target.

In [ ]:
video_path = Path("fig/rotation center trajectory.mp4")
run_video(video_path)

## **Phase 2.** Building Desired Trajectories for Trackers
<img src="fig/MPC decomposition scheme. Phase 2.png" 
        alt="Picture" 
        width="600"
        style="display: block; margin: 0 auto" />

For each virtual center's **predicted state** $\mathbf{\bar{x}}_{n} = [\bar{x}^{[I]}_{n}, \bar{y}^{[I]}_{n}, \bar{\psi}^{[I]}_{n}]^\intercal$ for $n \in \{k,\ldots, k+N-1\}$ and for each tracker $i \in \{1,2\}$
\begin{equation*}
    \mathbf{x}^{\text{tg}[i]}_{n} =
    \begin{bmatrix}
        x^{\text{tg}[i]}_{n} \\
        y^{\text{tg}[i]}_{n} \\
        \psi^{\text{tg}[i]}_{n}
    \end{bmatrix}
    =
    \begin{bmatrix}
        \bar{x}^{[I]}_{n} + R\cos{wt_n + \varphi_i} \\
        \bar{y}^{[I]}_{n} + R\sin{wt_n + \varphi_i} \\
        \psi^{\text{tg}[i]}_{n}
    \end{bmatrix}
\end{equation*}
where 
* $\psi^{\text{tg}[i]}_{n}$ is the angle of the tangent vector to the circumference at the point $[x^{\text{tg}[i]}_{n}, y^{\text{tg}[i]}_{n}]^\intercal$.
* $\varphi_1=0$ is a phase shift for the tracker 1,
* $\varphi_2 = \pi / 2$ is a phase shift for the tracker 2.

<img src="fig/desired trajectories.png" 
        alt="Picture" 
        width="600"
        style="display: block; margin: 0 auto" />

## **Phase 3.** Motion Planning for Trackers
<img src="fig/MPC decomposition scheme. Phase 3.png" 
        alt="Picture" 
        width="600"
        style="display: block; margin: 0 auto" />

### Optimal Control Probem Formulation $\mathcal{OCP2}$

\begin{equation*}
    \min_{\bar{\mathbf{u}}^{[i]}, \bar{\mathbf{x}}^{[i]}} J_\text{track}^{[D2]} + J_\text{velocity}^{[D2]} + J_\text{smooth}^{[D2]},
\end{equation*}

\begin{align*}
    J_\text{track}^{[D2]} &= \sum_{i=1}^{2} \sum_{n=k}^{k+N-1} (\mathbf{x}^{[i]}_n - \mathbf{x}^{\text{tg}[i]}_n)Q(\mathbf{x}^{[i]}_n - \mathbf{x}^{\text{tg}[i]}_n)^\intercal, \\
    J_\text{velocity}^{[D2]} &= \sum_{i=1}^{2} \sum_{n=k}^{k+N-1} (\mathbf{u}^{[i]}_n - \mathbf{u}^{\text{tg}})R(\mathbf{u}^{[i]}_n - \mathbf{u}^{\text{tg}})^\intercal \\
    J_\text{smooth}^{[D2]} &= \sum_{i=1}^{2} \sum_{n=k+1}^{k+N-1} (\mathbf{u}^{[i]}_n - \mathbf{u}^{[i]}_{n-1})D(\mathbf{u}^{[i]}_n - \mathbf{u}^{[i]}_{n-1})^\intercal, \\
\end{align*}

subject to
\begin{align*}
    \bar{\mathbf{x}}_{n+1}^{[i]} &= \mathbf{g_d} (\bar{\mathbf{x}}_{n}^{[i]}, \bar{\mathbf{u}}_{n}^{[I]}), \\
    ||\bar{\mathbf{p}}_{n}^{[1]}& - \bar{\mathbf{p}}_{n}^{[2]}|| > R_a, \\
    \bar{\mathbf{x}}_{k}^{[i]} &= \mathbf{x}^{[i]}, \\
    \bar{\mathbf{u}}_{n}^{[i]} &\in \mathcal{V}^{[i]},
\end{align*}
for $n \in \{k,\ldots, k+N-1\}$ and for each tracker $i \in \{1,2\}$

In [ ]:
video_path = Path(f"fig/tracker trajectories case 1.mp4")
run_video(video_path)

## Trackers control vector
<img src="fig/tracker controls case 1.png" 
        alt="Picture" 
        width="1200"
        style="display: block; margin: 0 auto" />

# Simulation Parameters

### Control Constraints
* Linear velocity $v^{[i]} \in [0, 4.0]$ m/s,
* Angular velocity $r^{[i]} \in [-\pi / 3, \pi / 3]$.

### MPC Parameters
* Weight matrix for maintaining trajectory: $Q = \text{diag}[15., 15., 0.1]$,
* Weight matrix for maintaining velocity: $R = \text{diag}[0.5, 0.02]$,
* Weight matrix for penalizing control difference $D = \text{diag}[0.5, 0.5]$,
* Prediction horizon $N=20$,
* Discrete time step $\Delta T = 0.1$ seconds.

### Additional Info
* **CasADi** open-source tool was used as NLP solver,
* Each NLP solution step takes approximately **30ms** (for both MPC controllers) -> Up to **30Hz** achievable.

## Cumulative Solution Histogram for the Circulation Trajectory
<img src="fig/cumulative trajectories case 1.png"
        alt="Picture" 
        width="750"
        style="display: block; margin: 0 auto" />

## Cumulative Solution Histogram for the Trajectory without Circulation
<img src="fig/cumulative trajectories case 2.png"
        alt="Picture" 
        width="750"
        style="display: block; margin: 0 auto" />

## Conclusions
* We have developed an MPC-based approach for target pursuit, enabling future localization.
* The efficiency of the proposed approach for the target localization task is demonstrated using the time cumulative histogram approach.

## Future Work
* The **SLAP** problem will be addressed for the case when the position and velocity of the target are unknown and must be estimated in real-time.
* We will test the approach with more in-depth simulation using full dynamic simulation of marine vehicles and position estimation noise.

# Thank you for attention

**Presenter**: Anton Tolstonogov

**Title**: TDoA-based Target Pursuit: A Model Predictive Control Approach

**Open in Google Colab**: https://colab.research.google.com/github/bioniwulf/DSO-FinalProject/blob/main/Notebook.ipynb

**QR code for online access to the project is below**
<img src="fig/qr_code.png"
        alt="Picture" 
        width="300"
        style="display: block; margin: 0px 0px 0px 0px left"
        />

# Appendix

## Different Weight matrices (maintaining trajectory)

### Reference value
* Weight matrix for maintaining trajectory: $Q = \text{diag}[15., 15., 0.1]$,
* Weight matrix for maintaining velocity: $R = \text{diag}[0.5, 0.02]$.

### Case 1
* Weight matrix for maintaining trajectory: $Q = \text{diag}[\mathbf{3.}, \mathbf{3.}, \mathbf{0.02}]$,
* Weight matrix for maintaining velocity: $R = \text{diag}[0.5, 0.02]$.

### Case 2
* Weight matrix for maintaining trajectory: $Q = \text{diag}[\mathbf{0.5}, \mathbf{0.5}, \mathbf{0.003}]$,
* Weight matrix for maintaining velocity: $R = \text{diag}[0.5, 0.02]$.

### Reference value
* Weight matrix for maintaining trajectory: $Q = \text{diag}[15., 15., 0.1]$,
* Weight matrix for maintaining velocity: $R = \text{diag}[0.5, 0.02]$.

<img src="fig/tracker controls case 1.png"
        alt="Picture" 
        width="1000"
        style="display: block; margin: 0px 0px 0px 0px center"
        />

### Case 1
* Weight matrix for maintaining trajectory: $Q = \text{diag}[\mathbf{3.}, \mathbf{3.}, \mathbf{0.02}]$,
* Weight matrix for maintaining velocity: $R = \text{diag}[0.5, 0.02]$.

<img src="fig/tracker controls case 1. WTC1.png"
        alt="Picture" 
        width="1000"
        style="display: block; margin: 0px 0px 0px 0px center"
        />

### Case 2
* Weight matrix for maintaining trajectory: $Q = \text{diag}[\mathbf{0.5}, \mathbf{0.5}, \mathbf{0.003}]$,
* Weight matrix for maintaining velocity: $R = \text{diag}[0.5, 0.02]$.

<img src="fig/tracker controls case 1. WTC2.png"
        alt="Picture" 
        width="1000"
        style="display: block; margin: 0px 0px 0px 0px center"
        />

## Different Weight matrices (Position vs Heading)

### Reference value
* Weight matrix for maintaining trajectory: $Q = \text{diag}[0.5, 0.5, 0.003]$
### Case 1
* Weight matrix for maintaining trajectory: $Q = \text{diag}[0.5, 0.5, \mathbf{0.05}]$
### Case 2
* Weight matrix for maintaining trajectory: $Q = \text{diag}[0.5, 0.5, \mathbf{0.1}]$

### Reference value
Weight matrix for maintaining trajectory: $Q = \text{diag}[0.5, 0.5, 0.003]$

<img src="fig/tracker controls case 1. WTC2.png"
        alt="Picture" 
        width="1000"
        style="display: block; margin: 0px 0px 0px 0px center"
        />

### Case 1
Weight matrix for maintaining trajectory: $Q = \text{diag}[0.5, 0.5, \mathbf{0.05}]$

<img src="fig/tracker controls case 1. WHC1.png"
        alt="Picture" 
        width="1000"
        style="display: block; margin: 0px 0px 0px 0px center"
        />

### Case 2
Weight matrix for maintaining trajectory: $Q = \text{diag}[0.5, 0.5, \mathbf{0.1}]$

<img src="fig/tracker controls case 1. WHC2.png"
        alt="Picture" 
        width="1000"
        style="display: block; margin: 0px 0px 0px 0px center"
        />

### Trackers' Trajectory (Case 2)
Weight matrix for maintaining trajectory: $Q = \text{diag}[0.5, 0.5, \mathbf{0.1}]$

<img src="fig/cumulative trajectories case 1. WHC2.png"
        alt="Picture" 
        width="1000"
        style="display: block; margin: 0px 0px 0px 0px center"
        />

## Different Prediction Horizon

### Case 1
* Prediction Horizon: **10 steps**, Calculation time: 17ms / step
### Reference value
* Prediction Horizon: 20 steps, Calculation time: 30ms / step
### Case 2
* Prediction Horizon: **30 steps**, Calculation time: 34ms / step
### Case 3
* Prediction Horizon: **50 steps**, Calculation time: 51ms / step

### Case 1
* Prediction Horizon: **10 steps**, Calculation time: 17ms / step

<img src="fig/tracker controls case 1. WPC1.png"
        alt="Picture" 
        width="1000"
        style="display: block; margin: 0px 0px 0px 0px center"
        />

### Reference value
* Prediction Horizon: 20 steps, Calculation time: 30ms / step

<img src="fig/tracker controls case 1. WTC2.png"
        alt="Picture" 
        width="1000"
        style="display: block; margin: 0px 0px 0px 0px center"
        />

### Case 2
* Prediction Horizon: **30 steps**, Calculation time: 34ms / step

<img src="fig/tracker controls case 1. WPC2.png"
        alt="Picture" 
        width="1000"
        style="display: block; margin: 0px 0px 0px 0px center"
        />

### Case 3
* Prediction Horizon: **50 steps**, Calculation time: 51ms / step

<img src="fig/tracker controls case 1. WPC3.png"
        alt="Picture" 
        width="1000"
        style="display: block; margin: 0px 0px 0px 0px center"
        />

### Trackers' Trajectory (Case 3)
* Prediction Horizon: **50 steps**, Calculation time: 51ms / step

<img src="fig/cumulative trajectories case 1. WPC3.png"
        alt="Picture" 
        width="1000"
        style="display: block; margin: 0px 0px 0px 0px center"
        />